In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

In [ ]:
#load borders
border = import_border("../data/border.json")
border = border 
# create the lattice grid
dx = 20 
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))
isgrid = isinside .| isborder
# define x and y coordinate for plotting
#x = repeat(reshape(xrange,1,Nx), Ny, 1)
#y = repeat(reshape(yrange,Ny,1), 1, Nx);
x = repeat(reshape(1:Nx,1,Nx), Ny, 1)
y = repeat(reshape(1:Ny,Ny,1), 1, Nx);

In [ ]:
#plot(border[:,1], border[:,2])
#scatter(x[isinside], y[isinside])

In [ ]:
bx = 2 * ones(Ny, Nx)
by = 2 * ones(Ny, Nx)

# adding
#bx .+= local_disturbance(isinside, xrange, yrange, [-200., 250.], 4E6, 500.)
#by .+= local_disturbance(isinside, xrange, yrange, [-200., 250.], 4E6, 500.)

#no static injection
p = zeros(Ny, Nx)

# one "punctual" generator and one load
#p = local_disturbance(isinside, xrange, yrange, [-1500.0, -750], 9.0, 100.)
#p .+= local_disturbance(isinside, xrange, yrange, [750.0, -250], -9.0, 100.)
contour(bx, fill=true)

In [ ]:
# compute the stable solution
th_ref, ths = compute_stable_sol(isinside, n, bx, by, p, Niter=14000, interval= 1000, tol=1E-9)
contour(th_ref, fill=true)

In [ ]:
# evolution for debugging
#ctr_movie(Float64.(1:size(ths,3)), ths)


In [ ]:
# perform a dynamical simulation
m = 1E-5*ones(Ny, Nx)
d = 0.6*copy(m)

m[.!isgrid] .= 0.0
d[.!isgrid] .= 0.0

# adding a fault
p .+= local_disturbance(isinside, xrange, yrange, [750.0, -250], -9.0, 100.)
ts, thetas, omegas = perform_dyn_sim(isinside, n, bx, by, p, m, d, th_ref, interval = 1000, Ndt = Int64(5e5), dt = 0.0001);

In [ ]:
# plotting the frequency a different locations
coord = [-1500.0 -750; 0.0 0.0; 750 -500; 50. -1050.]
time_plot(ts, omegas, coord, tstart = 0.0, tend=50.0)
current()

In [ ]:
# evolution of the frequency
ctr_movie(ts, thetas, tstart = 0.0, tend=50.0)

#trapz((yrange,xrange),p)/trapz((yrange,xrange),d) 

In [ ]:
heatmap(xrange, yrange, abs.(thetas[:,:,3]-th_ref))

In [ ]:
maximum(abs.(thetas[:,:,100]-th_ref))

In [ ]:
contour(1000*d,fill=true)